In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import chime
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
%load_ext chime

# Model selection
Kaggle Playground Series - Season 3, Episode 7

https://www.kaggle.com/competitions/playground-series-s3e7/overview

## Individual classifiers

### Note
I use the mean_test_score of cross validation in this notebook to compare model performance with different hyperparameters and see if the difference is big enough to continue optimization

# Data preparation

In [3]:
train = pd.read_csv('./data/train.csv', index_col='id'); train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
id,,,,,,,,,,,,,,,,,,
0,2,0,0,2,1,0,0,9,2018,1,14,1,1,11,0,67.50,0,0
1,2,0,1,2,0,0,0,117,2018,7,29,0,0,0,0,72.25,0,0
2,2,0,0,1,0,0,0,315,2018,12,2,0,0,0,0,52.00,0,0
3,1,0,0,2,1,0,0,32,2018,12,1,1,0,0,0,56.00,0,0
4,2,0,1,0,0,0,0,258,2018,10,16,0,0,0,0,100.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,3,0,0,4,0,0,1,160,2018,12,30,1,0,0,0,140.00,2,1
42096,2,0,0,3,0,0,0,34,2017,9,23,0,0,0,0,224.67,0,0
42097,2,0,0,2,2,0,0,292,2018,7,21,0,0,0,0,96.00,0,0


In [4]:
X_train = train.drop(columns='booking_status')
y_train = train.booking_status

In [5]:
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), 
                       columns=X_train.columns, 
                       index=X_train.index)
X_train

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
0,0.151040,-0.313454,-0.998814,-0.278849,1.294624,-0.160945,-0.515173,-1.170469,0.409661,-2.330398,-0.214091,0.428551,5.766758,33.699120,-0.101479,-0.998052,-0.737691
1,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,0.161740,0.409661,-0.209778,1.473487,-1.149928,-0.173408,-0.060506,-0.101479,-0.870153,-0.737691
2,0.151040,-0.313454,-0.998814,-0.979466,-0.407020,-0.160945,-0.515173,2.604122,0.409661,1.557404,-1.564154,-1.149928,-0.173408,-0.060506,-0.101479,-1.415406,-0.737691
3,-1.753925,-0.313454,-0.998814,-0.278849,1.294624,-0.160945,-0.515173,-0.886758,0.409661,1.557404,-1.676659,0.428551,-0.173408,-0.060506,-0.101479,-1.307702,-0.737691
4,0.151040,-0.313454,0.130259,-1.680084,-0.407020,-0.160945,-0.515173,1.901012,0.409661,0.850531,0.010919,-1.149928,-0.173408,-0.060506,-0.101479,-0.122955,-0.737691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,2.056005,-0.313454,-0.998814,1.122386,-0.407020,-0.160945,0.685890,0.692156,0.409661,1.557404,1.585992,0.428551,-0.173408,-0.060506,-0.101479,0.954088,1.842849
42096,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-0.862087,-2.441040,0.497095,0.798456,-1.149928,-0.173408,-0.060506,-0.101479,3.233919,-0.737691
42097,0.151040,-0.313454,-0.998814,-0.278849,2.996268,-0.160945,-0.515173,2.320411,0.409661,-0.209778,0.573445,-1.149928,-0.173408,-0.060506,-0.101479,-0.230659,-0.737691


# Best models
Assembling data from tuning individual classifiers

### Random forest

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
best_params_rf =\
{'max_depth': 40,
 'max_features': 'sqrt',
 'max_samples': 1.0,
 'min_samples_split': 40,
 'n_estimators': 2250}

In [8]:
random_forest = RandomForestClassifier(n_jobs=-1,
                                       random_state=8,
                                       bootstrap=True,
                                      **best_params_rf)

In [9]:
best_score_rf = 0.8911929904636795

### XGBoost

In [10]:
from xgboost import XGBClassifier

In [11]:
best_params_xgb =\
{'learning_rate': 0.3,
 'max_depth': 3,
 'n_estimators': 700,
 'reg_alpha': 0.0005,
 'reg_lambda': 0.2}

In [12]:
xgboost_clf = XGBClassifier(random_state=8,
                         max_leaves=0,
                         n_jobs=-1,
                         **best_params_xgb)

In [13]:
best_score_xgb = 0.8983333718679651

### KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
best_params_knn = {'n_neighbors': 60, 'p': 1, 'weights': 'uniform'}

In [16]:
knn = KNeighborsClassifier(algorithm='auto',
                           metric='minkowski',
                           n_jobs=-1,
                          **best_params_knn)

In [17]:
best_score_knn = 0.8636631176440875

### AdaBoost

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [19]:
best_params_base = {'max_depth': 25,
 'max_features': 'sqrt',
 'min_samples_split': 150}

In [20]:
best_params_ada = {'learning_rate': 0.0005, 'n_estimators': 900}

In [21]:
ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=8,
                                                                   **best_params_base),
                              random_state=8,
                              **best_params_ada)

In [22]:
best_score_ada = 0.8900577986404123

# Ensembles
## Voting classifier (unweighted)

In [27]:
from sklearn.ensemble import VotingClassifier

In [28]:
vote_with_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft'
)

In [29]:
%%time
%%chime
scores_with_knn = cross_val_score(vote_with_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


CPU times: user 5min 5s, sys: 7.19 s, total: 5min 12s
Wall time: 4min 54s


In [30]:
scores_with_knn.mean()

0.8959515851516204

In [31]:
scores_with_knn.std()

0.004247907076362921

In [32]:
vote_without_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('ada', ada)
    ],
    voting='soft'
)

In [33]:
%%time
%%chime
scores_without_knn = cross_val_score(vote_without_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


CPU times: user 4min 17s, sys: 1.81 s, total: 4min 19s
Wall time: 4min 27s


In [34]:
scores_without_knn.mean()

0.8981063043249012

In [35]:
scores_without_knn.std()

0.004297711993778989

## Voting classifier (weighted)

I use best scores of each classifier as its weight

In [36]:
weights_with_knn = [best_score_rf, best_score_xgb, best_score_knn, best_score_ada]

In [37]:
weighted_vote_with_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('knn', knn),
        ('ada', ada)
    ],
    voting='soft',
    weights=weights_with_knn
)

In [38]:
%%time
%%chime
weighted_scores_with_knn = cross_val_score(weighted_vote_with_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


CPU times: user 5min 5s, sys: 6.7 s, total: 5min 12s
Wall time: 4min 54s


In [39]:
weighted_scores_with_knn.mean()

0.8960693657674115

In [40]:
weighted_scores_with_knn.std()

0.004250088162659999

In [41]:
weights_without_knn = [best_score_rf, best_score_xgb, best_score_ada]

In [42]:
weighted_vote_without_knn = VotingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('ada', ada)
    ],
    voting='soft',
    weights=weights_without_knn
)

In [43]:
%%time
%%chime
weighted_scores_without_knn = cross_val_score(weighted_vote_without_knn,
                                       X_train, y_train,
                                       scoring='roc_auc',
                                       cv=5)


CPU times: user 4min 17s, sys: 1.87 s, total: 4min 19s
Wall time: 4min 25s


In [44]:
weighted_scores_without_knn.mean()

0.8981295739976263

In [45]:
weighted_scores_without_knn.std()

0.004297911144110737

### Conclusion on voting
- Voting improves results slightly compared to individual classifiers
- It is better to use top classifiers without KNN. For top classifiers, using their scores as weights does not result in significant improvement

## Prediction on the test set

In [46]:
test = pd.read_csv('./data/test.csv', index_col='id'); test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
42100,3,0,1,4,0,0,1,111,2018,12,5,1,0,0,0,115.60,2
42101,2,0,0,3,0,0,0,22,2017,10,21,0,0,0,0,85.00,0
42102,2,2,0,1,0,0,3,18,2018,8,10,1,0,0,0,240.00,1
42103,2,0,0,3,0,0,0,88,2018,5,30,0,0,0,0,80.75,0
42104,1,0,0,2,2,0,0,7,2018,9,21,1,0,0,0,144.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70163,2,0,1,2,0,0,0,315,2018,9,30,1,0,0,0,160.00,0
70164,2,0,2,2,0,0,0,81,2018,3,25,0,0,0,0,65.00,1
70165,2,0,1,2,0,0,0,40,2018,10,22,0,0,0,0,85.00,0


In [47]:
X_test = pd.DataFrame(scaler.transform(test), columns=test.columns, index=test.index)
X_test

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
id,,,,,,,,,,,,,,,,,
42100,2.056005,-0.313454,0.130259,1.122386,-0.407020,-0.160945,0.685890,0.087728,0.409661,1.557404,-1.226638,0.428551,-0.173408,-0.060506,-0.101479,0.297092,1.842849
42101,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-1.010111,-2.441040,0.850531,0.573445,-1.149928,-0.173408,-0.060506,-0.101479,-0.526846,-0.737691
42102,0.151040,4.129777,-0.998814,-0.979466,-0.407020,-0.160945,3.088017,-1.059452,0.409661,0.143658,-0.664112,0.428551,-0.173408,-0.060506,-0.101479,3.646696,0.552579
42103,0.151040,-0.313454,-0.998814,0.421768,-0.407020,-0.160945,-0.515173,-0.195983,0.409661,-0.916652,1.585992,-1.149928,-0.173408,-0.060506,-0.101479,-0.641282,-0.737691
42104,-1.753925,-0.313454,-0.998814,-0.278849,2.996268,-0.160945,-0.515173,-1.195140,0.409661,0.497095,0.573445,0.428551,-0.173408,-0.060506,-0.101479,1.061793,-0.737691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70163,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,2.604122,0.409661,0.497095,1.585992,0.428551,-0.173408,-0.060506,-0.101479,1.492610,-0.737691
70164,0.151040,-0.313454,1.259332,-0.278849,-0.407020,-0.160945,-0.515173,-0.282330,0.409661,-1.623525,1.023466,-1.149928,-0.173408,-0.060506,-0.101479,-1.065367,0.552579
70165,0.151040,-0.313454,0.130259,-0.278849,-0.407020,-0.160945,-0.515173,-0.788076,0.409661,0.850531,0.685950,-1.149928,-0.173408,-0.060506,-0.101479,-0.526846,-0.737691


In [50]:
weighted_vote_without_knn.fit(X_train, y_train)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=40,
                                                     max_features='sqrt',
                                                     max_samples=1.0,
                                                     min_samples_split=40,
                                                     n_estimators=2250,
                                                     n_jobs=-1,
                                                     random_state=8)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                         

In [51]:
y_pred = weighted_vote_without_knn.predict_proba(X_test)
y_pred

array([[0.91333583, 0.08666418],
       [0.9658149 , 0.03418511],
       [0.68422863, 0.31577136],
       ...,
       [0.90354732, 0.09645268],
       [0.41736209, 0.58263791],
       [0.04890227, 0.95109773]])

In [52]:
result = pd.DataFrame({'id': X_test.index, 'booking_status':y_pred[:, 1]})
result

,id,booking_status
0,42100,0.086664
1,42101,0.034185
2,42102,0.315771
3,42103,0.035767
4,42104,0.519960
...,...,...
28063,70163,0.950729
28064,70164,0.025368
28065,70165,0.096453
28066,70166,0.582638


In [53]:
result.to_csv('submit_weighted_vote_without_knn.csv', index=False)

## Stacking  classifiers

In [54]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

### Final estimator: logistic regression
With KNN

In [55]:
stack_1 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('knn', knn),
        ('ada', ada)
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    verbose=3
)

In [56]:
%%time
%%chime
scores_1 = cross_val_score(stack_1,
                           X_train, y_train,
                           scoring='roc_auc',
                           cv=5)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   13.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   16.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

CPU times: user 24min 35s, sys: 33.8 s, total: 25min 9s
Wall time: 23min 39s


In [57]:
scores_1.mean()

0.8998801693757397

In [58]:
scores_1.std()

0.004199336082174506

### Final estimator: logistic regression
Without KNN

In [59]:
stack_2 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('xgb', xgboost_clf),
        ('ada', ada)
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    verbose=3
)

In [60]:
%%time
%%chime
scores_2 = cross_val_score(stack_2,
                           X_train, y_train,
                           scoring='roc_auc',
                           cv=5)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   13.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   52.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concur

CPU times: user 21min 22s, sys: 9.88 s, total: 21min 32s
Wall time: 21min 49s


In [61]:
scores_2.mean()

0.8999567581948014

In [62]:
scores_2.std()

0.004239707904122041

### Final estimator: XGBoost

Without KNN

In [23]:
from sklearn.ensemble import StackingClassifier

In [24]:
stack_3 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('ada', ada)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

In [25]:
params = {
    'final_estimator__n_estimators': [50, 70, 90],
    'final_estimator__max_depth': [2, 4],
    'final_estimator__learning_rate': [0.1, 0.2],
    'final_estimator__reg_alpha': [0, 0.1, 0.2],
    'final_estimator__reg_lambda': [0, 0.1, 0.2]
}

In [26]:
model_3 = GridSearchCV(estimator=stack_3,
                     param_grid=params,
                     scoring='roc_auc',
                     cv = 4,
                    verbose = 3)

In [27]:
%%time
%%chime
model_3.fit(X_train, y_train)

Fitting 4 folds for each of 108 candidates, totalling 432 fits
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0;, score=0.890 total time= 3.8min
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0;, score=0.892 total time= 3.8min
[CV 3/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0;, score=0.893 total time= 3.8min
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0;, score=0.891 total time= 3.8min
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=50, 

[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0;, score=0.891 total time= 3.8min
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.1;, score=0.889 total time= 3.8min
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.1;, score=0.893 total time= 3.8min
[CV 3/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.1;, score=0.893 total time= 3.8min
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=70, final_estimator__reg_alpha=0, final_estimator__reg_lambda

[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=90, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.1;, score=0.891 total time= 3.8min
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=90, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.2;, score=0.890 total time= 3.8min
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=90, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.2;, score=0.892 total time= 3.8min
[CV 3/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=90, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.2;, score=0.893 total time= 3.8min
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=90, final_estimator__reg_alpha=0, final_estimator__reg_lamb

[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0, final_estimator__reg_lambda=0.2;, score=0.891 total time= 3.8min
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.889 total time= 3.8min
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.892 total time= 3.8min
[CV 3/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.892 total time= 3.8min
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_la

[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0;, score=0.891 total time= 3.8min
[CV 1/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.890 total time= 3.8min
[CV 2/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.892 total time= 3.8min
[CV 3/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.892 total time= 3.8min
[CV 4/4] END final_estimator__learning_rate=0.1, final_estimator__max_depth=4, final_estimator__n_estimators=70, final_estimator__reg_alpha=0.1, final_estimator__

KeyboardInterrupt: 

# Interrupted as it took too long to train

In [44]:
model_3.best_params_

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 70,
 'final_estimator__reg_alpha': 0.1,
 'final_estimator__reg_lambda': 0.1}

In [45]:
model_3.best_score_

0.9521756314451568

In [46]:
pd.Series(model_3.cv_results_['mean_test_score']).describe()

count    24.000000
mean      0.951557
std       0.000716
min       0.949249
25%       0.951262
50%       0.951865
75%       0.952046
max       0.952176
dtype: float64

### With KNN

In [49]:
stack_4 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('knn', knn),
        ('ada', ada)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

params = {
    'final_estimator__n_estimators': [50, 70, 100],
    'final_estimator__max_depth': [2, 4],
    'final_estimator__learning_rate': [0.05, 0.1, 0.2],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [58]:
params = {
    'final_estimator__n_estimators': [80, 100, 120, 150, 200],
    'final_estimator__max_depth': [2, 3],
    'final_estimator__learning_rate': [0.1],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [59]:
model_4 = GridSearchCV(estimator=stack_4,
                     param_grid=params,
                     scoring='roc_auc',
                     cv = 5,
                    verbose = 3)

In [60]:
%%time
%%chime
model_4.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.4s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.956 total time=  37.2s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.1s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.955 total time=  37.1s
[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_e

[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=120, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.1s
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.3s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.955 total time=  37.2s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.5s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=3, final_estimator__n_estimators=150, final_estimator__reg_alpha=0.1, final_esti

In [61]:
model_4.best_params_

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 80,
 'final_estimator__reg_alpha': 0.1,
 'final_estimator__reg_lambda': 0.1}

In [62]:
model_4.best_score_

0.9531706708568304

In [63]:
pd.Series(model_4.cv_results_['mean_test_score']).describe()

count    10.000000
mean      0.952920
std       0.000258
min       0.952283
25%       0.952868
50%       0.952964
75%       0.953087
max       0.953171
dtype: float64

### Results so far
The best result is achieved when KNN is included in a stacked classifier with XGBoost as the final estimator, although it is computationally expensive. I will try to add logistic regression as well

### With KNN and logistic regression

In [65]:
from sklearn.linear_model import LogisticRegression

In [66]:
best_params_lr = {'C': 0.1, 'max_iter': 50, 'penalty': 'l2'}
lr = LogisticRegression(class_weight='balanced',
                        random_state=8,
                        solver='liblinear',
                        **best_params_lr)

In [67]:
stack_5 = StackingClassifier(
    estimators = [
        ('rf', random_forest),
        ('knn', knn),
        ('ada', ada),
        ('lr', lr)
    ],
    final_estimator=XGBClassifier(random_state=8,
                                 max_leaves=0,
                                 n_jobs=-1),
    cv=5,
)

In [68]:
params = {
    'final_estimator__n_estimators': [50, 80, 100],
    'final_estimator__max_depth': [2, 4],
    'final_estimator__learning_rate': [0.01, 0.1, 0.5],
    'final_estimator__reg_alpha': [0.1],
    'final_estimator__reg_lambda': [0.1]
}

In [69]:
model_5 = GridSearchCV(estimator=stack_5,
                     param_grid=params,
                     scoring='roc_auc',
                     cv = 5,
                    verbose = 3)

In [70]:
%%time
%%chime
model_5.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.947 total time=  39.0s
[CV 2/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.6s
[CV 3/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.947 total time=  37.3s
[CV 4/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.949 total time=  37.5s
[CV 5/5] END final_estimator__learning_rate=0.01, final_estimator__max_depth=2, final_estimator

[CV 5/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.5s
[CV 1/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.4s
[CV 2/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.956 total time=  37.4s
[CV 3/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.952 total time=  37.5s
[CV 4/5] END final_estimator__learning_rate=0.1, final_estimator__max_depth=2, final_estimator__n_estimators=100, final_estimator__reg_alpha=0.1, final_estim

[CV 4/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.951 total time=  37.6s
[CV 5/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=50, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.946 total time=  37.4s
[CV 1/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.944 total time=  37.5s
[CV 2/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator__reg_lambda=0.1;, score=0.949 total time=  38.0s
[CV 3/5] END final_estimator__learning_rate=0.5, final_estimator__max_depth=4, final_estimator__n_estimators=80, final_estimator__reg_alpha=0.1, final_estimator

In [71]:
model_5.best_params_

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__n_estimators': 80,
 'final_estimator__reg_alpha': 0.1,
 'final_estimator__reg_lambda': 0.1}

In [72]:
model_5.best_score_

0.9531468487214768

In [73]:
pd.Series(model_5.cv_results_['mean_test_score']).describe()

count    18.000000
mean      0.950803
std       0.002509
min       0.945295
25%       0.949192
50%       0.951922
75%       0.952916
max       0.953147
dtype: float64

Addition of logistic regression did not improve results

# Conclusion
The best result is achieved when KNN is included in a stacked classifier with XGBoost as the final estimator, although it is computationally expensive.